In [1]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

In [2]:
sc = pyspark.SparkContext()

23/04/11 13:34:50 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/04/11 13:34:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/11 13:34:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sqlContext = SQLContext(sc)

/home/hadoop/anaconda3/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [18]:
# predictions = sqlContext\
#             .read.load('file:///home/hadoop/big-data-processing/04_Machine_learning_with_Big_Data/predictions/part-00000-a44b280d-5e28-42cf-bd15-35306406af82-c000.csv', 
#                           format='com.databricks.spark.csv', 
#                           header='true',inferSchema='true')
# predictions = sqlContext\
#             .read.load('file:///home/hadoop/big-data-processing/04_Machine_learning_with_Big_Data/predictions.csv/*.csv', 
#                           format='com.databricks.spark.csv', 
#                           header='true',inferSchema='true')
predictions = sqlContext\
            .read.load('file:///home/hadoop/big-data-processing/04_Machine_learning_with_Big_Data/predictions.csv', 
                          format='com.databricks.spark.csv', 
                          header='true',inferSchema='true')

In [19]:
predictions.show(5)

+----------+-----+
|prediction|label|
+----------+-----+
|       1.0|  1.0|
|       1.0|  1.0|
|       0.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
+----------+-----+
only showing top 5 rows



create an instance of MulticlassClassificationEvaluator to determine the accuracy of the predictions:

https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html

In [22]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="precisionByLabel")

The first two arguments specify the names of the label and prediction columns, and the third argument specifies that we want the overall precision.

In [28]:
precision = evaluator.evaluate(predictions)
print("Precision = %g " % (precision))

Precision = 0.756522 


In [30]:
evaluator2 = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
Accuracy = evaluator2.evaluate(predictions)
print("Accuracy = %g " % (Accuracy))

Accuracy = 0.784404 


In [40]:
print ("Accuracy = %.4g" % (Accuracy * 100))

Accuracy = 78.44


In [41]:
print ("Error = %g " % (1.0 - Accuracy))

Error = 0.215596 


**Display confusion matrix.** \
The MulticlassMetrics class can be used to generate a confusion matrix of our classifier model. However, unlike MulticlassClassificationEvaluator, MulticlassMetrics works with RDDs of numbers and not DataFrames, so we need to convert our predictions DataFrame into an RDD.

If we use the rdd attribute of predictions, we see this is an RDD of Rows:

In [31]:
predictions.rdd.take(2)

[Row(prediction=1.0, label=1.0), Row(prediction=1.0, label=1.0)]

Instead, we can map the RDD to tuple to get an RDD of numbers:

In [32]:
predictions.rdd.map(tuple).take(2)

[(1.0, 1.0), (1.0, 1.0)]

In [33]:
metrics = MulticlassMetrics(predictions.rdd.map(tuple))

/home/hadoop/anaconda3/lib/python3.10/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [34]:
metrics.confusionMatrix().toArray().transpose()

array([[87., 28.],
       [19., 84.]])

In [42]:
metrics.confusionMatrix().toArray()

array([[87., 19.],
       [28., 84.]])